# Read fire response from austraits data

We will download data from [AusTraits](https://austraits.org/) ([pre-print](https://www.biorxiv.org/content/10.1101/2021.01.04.425314v1)) and add entries to the database for resprouting time for each species.

Let's start loading the libraries

In [1]:
from pathlib import Path
import os
import json
import urllib
from zipfile import ZipFile
import pandas as pd
import numpy as np

## Read _austraits_ data 
We will download the file from the [Zenodo repository](https://zenodo.org/record/5112001) using the API url and saving this under the data folder.

In [2]:
repodir = Path("../") 
dataset = "https://zenodo.org/api/records/3568417"
outputdir = repodir / "data/austraits/"

if not os.path.isdir(outputdir):
    os.makedirs(outputdir)

We use urllib to open the url and read the data (if successfully connected!)

In [3]:
def getResponse(url):
    operUrl = urllib.request.urlopen(url)
    if(operUrl.getcode()==200):
       data = operUrl.read()
    else:
       print("Error receiving data", operUrl.getcode())
    return data
zrecord = getResponse(dataset)

Response data is in json format, need to parse it and read list of files:

In [4]:
jsonData = json.loads(zrecord)
#jsonData
for files in jsonData['files']:
    print(files['key'])

austraits-3.0.2.rds
austraits-3.0.2.zip
dictionary.html
NEWS.md
readme.txt


We want to download the zip file with the csv_files

In [5]:
outputfile = outputdir / jsonData['files'][1]['key']

if os.path.isfile(outputfile):
    print('File exists')
else:
    resp = getResponse(jsonData['files'][1]['links']['self'])
    output = open(outputfile,'wb')
    output.write(resp)
    output.close()

File exists


We will read from the zipfile the data that we need:

In [6]:
zfobj = ZipFile(outputfile)
zfobj.namelist()

['austraits-3.0.2/',
 'austraits-3.0.2/taxa.csv',
 'austraits-3.0.2/methods.csv',
 'austraits-3.0.2/definitions.yml',
 'austraits-3.0.2/build_info.md',
 'austraits-3.0.2/contributors.csv',
 'austraits-3.0.2/contexts.csv',
 'austraits-3.0.2/excluded_data.csv',
 'austraits-3.0.2/traits.csv',
 'austraits-3.0.2/taxonomic_updates.csv',
 'austraits-3.0.2/sites.csv',
 'austraits-3.0.2/sources.bib']

### Read files
We will need to read the files with the definitions (in _yaml_ format), the sources or references (in _bibtex_ format) and the traits and taxonomic data (in _csv_ format)

In [8]:
import yaml

with zfobj.open('austraits-3.0.2/definitions.yml') as file:
    try:
        ATdefinitions = yaml.safe_load(file)   
        print(ATdefinitions.keys())
    except yaml.YAMLError as exc:
        print(exc)

dict_keys(['traits', 'value_type', 'austraits', 'metadata'])


In [9]:
from pybtex.database.input import bibtex
parser = bibtex.Parser()

ATrefs = parser.parse_bytes(zfobj.open('austraits-3.0.2/sources.bib').read())


In [10]:
ATtraits = pd.read_csv(zfobj.open('austraits-3.0.2/traits.csv'),low_memory=False)

In [11]:
ATtaxa = pd.read_csv(zfobj.open('austraits-3.0.2/taxa.csv'))

We will also read the updated species data from BioNET:

In [12]:
repodir = Path("../") 
inputdir = repodir / "data/"
BioNET = pd.read_excel(inputdir / 'vis-survey-datasheet-6000.PowerQuery.20210708.xlsx')

## Read fire response data

In [13]:
ss = (ATtraits['trait_name']=='fire_response' )
ATtraits[ss]


,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
15332,Baker_2019,Commersonia bartramia,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_01,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Commersonia bartramia
15334,Baker_2019,Denhamia celastroides,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_02,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Denhamia celastroides
15336,Baker_2019,Blechnum neohollandicum,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_03,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Doodia aspera
15338,Baker_2019,Pittosporum undulatum,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_04,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Pittosporum undulatum
15340,Baker_2019,Polyscias sambucifolia,Main Arm -28.4936667_and_153.3923611,NaN,Baker_2019_05,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Polyscias sambucifolia
...,...,...,...,...,...,...,...,...,...,...,...,...
959253,White_2020,Roepera ovata,NaN,NaN,White_2020_7923,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum ovatum
959272,White_2020,Roepera prismatotheca,NaN,NaN,White_2020_7924,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum prismatothecum
959291,White_2020,Roepera similis,NaN,NaN,White_2020_7925,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum simile
959310,White_2020,Roepera ammophila,NaN,NaN,White_2020_7926,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum sp. aff. ammophilum


In [14]:
def extract_reflabel(refid):
    authors=list()
    year=ATrefs.entries[refid].fields['year']
    for person in ATrefs.entries[refid].persons['author']:
        authors.extend(person.last_names)
    reflabel = "%s %s" % (" ".join(authors),year)
    if len(reflabel)>50:
        reflabel=reflabel[0:47]+"..."
    return(reflabel)

def extract_refinfo(refid):
    year=ATrefs.entries[refid].fields['year']
    title=ATrefs.entries[refid].fields['title']
    persons = ATrefs.entries[refid].persons['author']
    if len(persons)==1:
        refcitation = "%s (%s) %s" % (persons[0],year, title)
    else:
        authors=list()
        for person in persons:
            authors.append(person.__str__())
        refcitation = "%s (%s) %s" % ("; ".join(authors),year, title)
    for f in ('journal','volume','doi'):
        if f in ATrefs.entries[refid].fields.keys():
            refcitation = refcitation + " " + ATrefs.entries[refid].fields[f]
    return refcitation 

def match_spcode(row):
    spname=row['taxon_name']
    altname=row['original_name']
    result={'species':spname}
    if altname!=spname:
        result['original_notes']=['original_name:',altname]
    spp_info = BioNET[BioNET['scientificName'] == spname] 
    spcode=None
    if len(spp_info)==1 and spp_info.speciesCode_Synonym is not None:
        spcode=spp_info.speciesCode_Synonym.values[0]
        result['species_code']=spcode
    elif spname != altname:
        spp_info = BioNET[BioNET['scientificName'] == altname]
        if len(spp_info)==1 and spp_info.speciesCode_Synonym is not None:
            spcode=spp_info.speciesCode_Synonym.values[0]
            result['species_code']=spcode
            result['original_notes'].append('original name used to match with BioNET names')
 
    return result


In [17]:
print(extract_refinfo('NSWFRD_2014'))
print(extract_reflabel('NSWFRD_2014'))

Kenny, Belinda; Orscheg, Corinna; Tasker, Elizabeth; Gill, Malcolm A.; Bradstock, Ross (2014) {NSW Flora Fire Response Database, v2.1}
Kenny Orscheg Tasker Gill Bradstock 2014


In [18]:
def create_record(row):
    refid=row['dataset_id']
    reflabel = extract_reflabel(refid)
    transvalue=switcher.get(row['value'], None)
    
    record={'main_source': 'austraits-3.0.2',
            'additional_notes': ['Values reclassified by JRFP',
                                'Automatic extraction with python script'],
            'raw_value': [row['trait_name'],row['value'],row['value_type']],
            'original_notes': list(),
           'original_sources':[reflabel]}
    spinfo=match_spcode(row)
    for key in spinfo.keys():
        record[key]=spinfo[key]
    if reflabel=='NSWFRD_2014':
        record['weight'] = 0
        record['weight_notes'] = ["python-script import","default of 0 for redundant records"]
    else:
        record['weight'] = 1
        record['weight_notes'] = ["python-script import","default of 1"]
    if transvalue is not None:   
        record["norm_value"]=transvalue
    if row['site_name'] != "nan":
        record['original_notes'].append('site name:')
        record['original_notes'].append(row['site_name'])
    return(record)

In [21]:
target = ATtraits[ss].head()

switcher={
        "fire_killed": "None",
        'not_fire_killed_does_not_resprout': "None",
        'fire_not_relevant': 'Unknown', 
        'fire_killed resprouts': 'Half',
        'unknown': 'Unknown',
        "resprouts": "All"
    }
    
target.fillna("nan",inplace=True)
reflist=list()
records=list()
for idx, row in target.iterrows():
    record=create_record(row)
    refid=row['dataset_id']
    extract_reflabel(refid)
    if refid not in reflist:
        reflist.append(refid)
    records.append(record)
records

[{'main_source': 'austraits-3.0.2',
  'additional_notes': ['Values reclassified by JRFP',
   'Automatic extraction with python script'],
  'raw_value': ['fire_response', 'resprouts', 'expert_mean'],
  'original_notes': ['site name:', 'Bogangar -28.3303611_and_153.5519444'],
  'original_sources': ['Baker 2019'],
  'species': 'Commersonia bartramia',
  'species_code': '6129',
  'weight': 1,
  'weight_notes': ['python-script import', 'default of 1'],
  'norm_value': 'All'},
 {'main_source': 'austraits-3.0.2',
  'additional_notes': ['Values reclassified by JRFP',
   'Automatic extraction with python script'],
  'raw_value': ['fire_response', 'resprouts', 'expert_mean'],
  'original_notes': ['site name:', 'Bogangar -28.3303611_and_153.5519444'],
  'original_sources': ['Baker 2019'],
  'species': 'Denhamia celastroides',
  'species_code': '8387',
  'weight': 1,
  'weight_notes': ['python-script import', 'default of 1'],
  'norm_value': 'All'},
 {'main_source': 'austraits-3.0.2',
  'additiona

In [22]:
target[0:]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
15332,Baker_2019,Commersonia bartramia,Bogangar -28.3303611_and_153.5519444,nan,Baker_2019_01,fire_response,resprouts,nan,nan,expert_mean,nan,Commersonia bartramia
15334,Baker_2019,Denhamia celastroides,Bogangar -28.3303611_and_153.5519444,nan,Baker_2019_02,fire_response,resprouts,nan,nan,expert_mean,nan,Denhamia celastroides
15336,Baker_2019,Blechnum neohollandicum,Bogangar -28.3303611_and_153.5519444,nan,Baker_2019_03,fire_response,resprouts,nan,nan,expert_mean,nan,Doodia aspera
15338,Baker_2019,Pittosporum undulatum,Bogangar -28.3303611_and_153.5519444,nan,Baker_2019_04,fire_response,resprouts,nan,nan,expert_mean,nan,Pittosporum undulatum
15340,Baker_2019,Polyscias sambucifolia,Main Arm -28.4936667_and_153.3923611,nan,Baker_2019_05,fire_response,resprouts,nan,nan,expert_mean,nan,Polyscias sambucifolia


In [23]:
from configparser import ConfigParser
import psycopg2
from psycopg2.extensions import AsIs

filename = repodir / 'secrets' / 'database.ini'
section = 'aws-lght-sl'

parser = ConfigParser()
parser.read(filename)

dbparams = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        dbparams[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

Exception: Section aws-lght-sl not found in the ../secrets/database.ini file

In [253]:
#split your dataframe into smaller dataframes contained in a list.

ss = (ATtraits['trait_name']=='fire_response' )
df = ATtraits[ss]
df.fillna("nan",inplace=True)
n = 250  #chunk row size
list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]

len(list_df)

/var/folders/14/vyrp_30975d7y17lqxjblf2c0000gn/T/ipykernel_7127/1163626233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna("nan",inplace=True)


74

In [254]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**dbparams)
cur = conn.cursor()
affected_rows=0
switcher={
        "fire_killed": "None",
        'not_fire_killed_does_not_resprout': "None",
        'fire_not_relevant': 'Unknown', 
        'fire_killed resprouts': 'Half',
        'unknown': 'Unknown',
        "resprouts": "All"
    }

for target in list_df:    
    reflist=list()
    records=list()
    for idx, row in target.iterrows():
        record=create_record(row)
        refid=row['dataset_id']
        extract_reflabel(refid)
        if refid not in reflist:
            reflist.append(refid)
        records.append(record)
        
    for refid in reflist:
        cur.execute("INSERT INTO litrev.ref_list(ref_code,alt_code,ref_cite) values(%s,%s,%s) ON CONFLICT DO NOTHING",
                    (extract_reflabel(refid), refid, extract_refinfo(refid)))
        affected_rows = affected_rows+cur.rowcount
    conn.commit()
    print("total number of lines updated: %s" % affected_rows)

    insert_statement = 'insert into litrev.surv1 (%s) values %s ON CONFLICT DO NOTHING'
    print("total of %s records prepared" % len(records)) 
    for record in records: 
        cur.execute(insert_statement, (AsIs(','.join(record.keys())), tuple(record.values())))
        affected_rows = affected_rows+cur.rowcount
    records.clear()
    conn.commit()
    print("total number of lines updated: %s" % affected_rows)

cur.close()
if conn is not None:
    conn.close()
    print('Database connection closed.')     


Connecting to the PostgreSQL database...
total number of lines updated: 0
total of 250 records prepared
total number of lines updated: 250
total number of lines updated: 250
total of 250 records prepared
total number of lines updated: 500
total number of lines updated: 500
total of 250 records prepared
total number of lines updated: 750
total number of lines updated: 750
total of 250 records prepared
total number of lines updated: 1000
total number of lines updated: 1000
total of 250 records prepared
total number of lines updated: 1250
total number of lines updated: 1251
total of 250 records prepared
total number of lines updated: 1501
total number of lines updated: 1501
total of 250 records prepared
total number of lines updated: 1751
total number of lines updated: 1751
total of 250 records prepared
total number of lines updated: 2001
total number of lines updated: 2001
total of 250 records prepared
total number of lines updated: 2251
total number of lines updated: 2251
total of 250 r

In [16]:
print(ATdefinitions['traits']['elements']['fire_response'].keys())

dict_keys(['description', 'type', 'label', 'values'])


In [17]:
print(ATdefinitions['traits']['elements']['fire_response']['values'])

{'fire_killed': 'Plants killed by hot fires', 'resprouts': "Plants resprout from underground storage organ following fire. (For studies that don't differentiate between respouting strength)", 'not_fire_killed_does_not_resprout': 'Plants that are rarely killed by a moderate-intensity fire, but do not resprout', 'fire_not_relevant': 'Plant never affected by fire (for aquatic taxon)', 'unknown': 'Fire status assessed, but unknown'}


### References

In [117]:
#bib_data.entries.keys()
for key in bib_data.entries.keys():
    print(key)
bib_data.entries['Adams_1984']
# dir(bib_data.entries['Adams_1984'])

Adams_1984
Ahrens_2019
ANBG_2019
Angevin_2011
Apgaua_2015
Apgaua_2017
Ashton_1975
Ashton_1976
Atkinson_2020
Atkinson_2020_2
Attiwill_1980
Baker_2019
Barlow_1981
Bean_1997
Bell_1985
Bennett_1997
Bevege_1978
Birk_1992
Blackman_2010
Blackman_2014
Blackman_2018
Bloomfield_2018
Bolza_1975
Bragg_2002
BRAIN_2007
Briggs_2010
Brock_1993
Brodribb_2009
Buckton_2019
Burgess_2007
Burrows_2001
Butler_2011
CAB_2009
Caldwell_2016
Canham_2009
Carpenter_1994
Carpenter_2005
Catford_2011
Catford_2014
Cernusak_2006
Cernusak_2011
Chandler_2002
Chave_2009
Cheal_2017
Cheesman_2020
Chen_2017
Chinnock_2007
Choat_2005
Choat_2006
Choat_2012
Chudnoff_1984
CIRAD_2009
Clarke_2015
Cooper_2004
Cooper_2013
Cornwell_2006
CPBR_2002
Craven_1987
Craven_2010
Crisp_2017
Cromer_1975
Cross_2009
Crous_2013
Crous_2019
Cunningham_1999
Curran_2009
Curtis_2012
Denton_2007
Desh_1996
Detombeur_2021
Dong_2017
Dong_2020
Du_2018
Du_2019
Duan_2015
Duncan_1998
Duncan_2011
Dwyer_2017
Dwyer_2018
Eamus_1998
Eamus_1999
Eamus_1999_2
Edwards_20

Entry('article',
  fields=[
    ('year', '1984'), 
    ('journal', 'Australian Journal of Botany'), 
    ('title', '{Role of Acacia spp. in nutrient balance and cycling in regenerating Eucalyptus regnans F. Muell. forests. I. Temporal changes in biomass and nutrient content}'), 
    ('volume', '32'), 
    ('number', '2'), 
    ('pages', '205--215'), 
    ('doi', '10.1071/bt9840205')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Adams, M. A.'), Person('M, P.'), Person('{Attiwill}')])]))

In [176]:
print(bib_data.entries['Adams_1984'].fields['year'])
##''.join(person for person in bib_data.entries['Adams_1984'].persons['author'])
for person in bib_data.entries['Adams_1984'].persons['author']:
    print(person.__str__())

dir(person)


1984
Adams, M. A.
M, P.
{Attiwill}


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parse_string',
 'bibtex_first',
 'bibtex_first_names',
 'first',
 'first_names',
 'get_part',
 'get_part_as_text',
 'last',
 'last_names',
 'lineage',
 'lineage_names',
 'middle',
 'middle_names',
 'prelast',
 'prelast_names',
 'rich_first_names',
 'rich_last_names',
 'rich_lineage_names',
 'rich_middle_names',
 'rich_prelast_names',
 'style1_re',
 'style2_re',
 'valid_roles']

### Traits data

In [23]:
sppname='Actinotus helianthi'
traits=('fire_response_juvenile','fire_response','fire_cued_seeding')

ss = (ATtraits['taxon_name']==sppname) & (ATtraits['trait_name'].isin(traits))
ATtraits[ss]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
217795,Falster_2005_2,Actinotus helianthi,Myall_Lakes,NaN,Falster_2005_2_05,fire_response,fire_killed,NaN,2002-09,expert_mean,NaN,Actinotus helianthi
454654,NSWFRD_2014,Actinotus helianthi,NaN,NaN,NSWFRD_2014_0177,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Actinotus helianthi


### Taxonomic data

We will read this into a pandas data frame:

Check information from one species

In [25]:
target = df[df['taxon_name'] == "Actinotus helianthi"] 
target

,taxon_name,source,acceptedNameUsageID,scientificNameAuthorship,taxonRank,taxonomicStatus,family,taxonDistribution,ccAttributionIRI,genus
1665,Actinotus helianthi,APC,https://id.biodiversity.org.au/node/apni/2895645,Labill.,Species,accepted,Apiaceae,"Qld, NSW, Vic (naturalised)",https://id.biodiversity.org.au/tree/51354547/5...,Actinotus


In [26]:
spp_info = CAPS[CAPS['scientificName'] == "Actinotus helianthi"] 
spp_info.transpose()

,3
speciesID,2361
taxonRank,Species
classID,38
class,Flora
orderID,137
order,Flora
familyID,157
family,Apiaceae
sortOrder,6031
genusID,1823.0


In [27]:
qry = target['acceptedNameUsageID'].values[0]+".json"
qry
#apniData=json.loads(qry) # error?

'https://id.biodiversity.org.au/node/apni/2895645.json'

In [28]:
apniData = json.loads(getResponse(qry))
apniData

{'treeElement': {'class': 'au.org.biodiversity.nsl.TreeElement',
  '_links': {'elementLink': 'https://id.biodiversity.org.au/tree/51662411/51242199',
   'taxonLink': 'https://id.biodiversity.org.au/node/apni/2895645',
   'parentElementLink': 'https://id.biodiversity.org.au/tree/51662411/51375672',
   'nameLink': 'https://id.biodiversity.org.au/name/apni/75081',
   'instanceLink': 'https://id.biodiversity.org.au/instance/apni/755469',
   'sourceElementLink': None},
  'tree': {'class': 'au.org.biodiversity.nsl.Tree',
   '_links': {'permalinks': [{'link': 'https://id.biodiversity.org.au/tree/apni/APC',
      'preferred': True,
      'resources': 1}]},
   'audit': None,
   'name': 'APC'},
  'simpleName': 'Actinotus helianthi',
  'namePath': 'Plantae/Charophyta/Equisetopsida/Magnoliidae/Asteranae/Apiales/Apiaceae/Actinotus/helianthi',
  'treePath': '/51209397/51209398/51209399/51210622/51236316/51241866/51242182/51375672/51242199',
  'displayHtml': '<data><scientific><name data-id=\'75081\'

## Explore traits

## Species list from NSW

The previous version of the CAPS (whatever that means) table was available from this (non-permanent?) url:

But we got an updated version from a BioNET export prepared by Renee:

In [4]:
CAPS

,speciesID,taxonRank,classID,class,orderID,order,familyID,family,sortOrder,genusID,...,stateConservation,protectedInNSW,sensitivityClass,TSProfileID,countryConservation,highThreatWeed,widelyCultivatedNativeSpecies,CAMBA,JAMBA,ROKAMBA
0,2358,Species,38,Flora,137,Flora,155,Brassicaceae,7878,3019.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
1,2359,Species,38,Flora,137,Flora,156,Myrtaceae,13245,2638.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
2,2360,Species,38,Flora,137,Flora,156,Myrtaceae,13447,3165.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
3,2361,Species,38,Flora,137,Flora,157,Apiaceae,6031,1823.0,...,Not Listed,True,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
4,2362,Species,38,Flora,137,Flora,157,Apiaceae,6046,1944.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15727,24289,Species,38,Flora,137,Flora,170,Orchidaceae,14157,3583.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
15728,24291,Species,38,Flora,137,Flora,203,Araliaceae,6378,3931.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
15729,24292,Species,38,Flora,137,Flora,214,Pittosporaceae,14710,3474.0,...,Critically Endangered,False,Not Sensitive,20378.0,Not Listed,NaN,NaN,False,False,False
15730,24293,Species,38,Flora,137,Flora,206,Malvaceae,12327,1825.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False


In [5]:
CAPS.columns

Index(['speciesID', 'taxonRank', 'classID', 'class', 'orderID', 'order',
       'familyID', 'family', 'sortOrder', 'genusID', 'genus',
       'parentSpeciesID', 'specificEpithet', 'infraspecificEpithet',
       'scientificNameAuthorship', 'scientificNameID', 'speciesCode_Synonym',
       'scientificName', 'scientificNameHTML', 'vernacularName',
       'otherVernacularNames', 'taxonID', 'currentScientificNameCode',
       'currentScientificName', 'currentVernacularName', 'isCurrent',
       'generalTypeID', 'generalType', 'establishmentMeans',
       'primaryGrowthForm', 'primaryGrowthFormGroup', 'secondaryGrowthForms',
       'secondaryGrowthFormGroups', 'stateConservation', 'protectedInNSW',
       'sensitivityClass', 'TSProfileID', 'countryConservation',
       'highThreatWeed', 'widelyCultivatedNativeSpecies', 'CAMBA', 'JAMBA',
       'ROKAMBA'],
      dtype='object')

In [6]:
target = CAPS[CAPS['scientificName'] == "Actinotus helianthi"] 
target

,speciesID,taxonRank,classID,class,orderID,order,familyID,family,sortOrder,genusID,...,stateConservation,protectedInNSW,sensitivityClass,TSProfileID,countryConservation,highThreatWeed,widelyCultivatedNativeSpecies,CAMBA,JAMBA,ROKAMBA
3,2361,Species,38,Flora,137,Flora,157,Apiaceae,6031,1823.0,...,Not Listed,True,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False


In [29]:
ATtraits['trait_name'].unique()

array(['leaf_area', 'leaf_delta13C', 'leaf_N_per_dry_mass', 'leaf_PRI',
       'modified_NDVI', 'specific_leaf_area', 'water_band_index',
       'wood_density', 'seed_length', 'seed_shape', 'seed_texture',
       'seed_width', 'seed_mass', 'germination_treatment',
       'dispersal_appendage', 'dispersal_syndrome',
       'leaf_dry_matter_content', 'plant_growth_form', 'plant_height',
       'leaf_length', 'leaf_thickness', 'leaf_width',
       'sapwood_specific_conductivity_theoretical', 'vessel_density',
       'vessel_diameter', 'vessel_lumen_fraction',
       'vessel_diameter_hydraulic', 'vessel_multiple_fraction',
       'water_use_efficiency_intrinsic', 'leaf_hairs_adult', 'leaf_shape',
       'fire_response', 'regen_strategy', 'fire_cued_seeding',
       'leaf_compoundness', 'seed_breadth', 'leaf_phenology',
       'flowering_time', 'life_history', 'leaf_type',
       'leaf_hydraulic_conductivity', 'leaf_hydraulic_vulnerability',
       'leaf_turgor_loss_point', 'ci_at_Amax', 'c

In [30]:
ss = (ATtraits['trait_name']=='resprouting_proportion_individuals' )
ATtraits[ss]


,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
15332,Baker_2019,Commersonia bartramia,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_01,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Commersonia bartramia
15334,Baker_2019,Denhamia celastroides,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_02,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Denhamia celastroides
15336,Baker_2019,Blechnum neohollandicum,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_03,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Doodia aspera
15338,Baker_2019,Pittosporum undulatum,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_04,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Pittosporum undulatum
15340,Baker_2019,Polyscias sambucifolia,Main Arm -28.4936667_and_153.3923611,NaN,Baker_2019_05,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Polyscias sambucifolia
...,...,...,...,...,...,...,...,...,...,...,...,...
959253,White_2020,Roepera ovata,NaN,NaN,White_2020_7923,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum ovatum
959272,White_2020,Roepera prismatotheca,NaN,NaN,White_2020_7924,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum prismatothecum
959291,White_2020,Roepera similis,NaN,NaN,White_2020_7925,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum simile
959310,White_2020,Roepera ammophila,NaN,NaN,White_2020_7926,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum sp. aff. ammophilum


In [31]:
ATtraits[ss]['value'].unique()

array(['resprouts', 'fire_killed', 'unknown', 'fire_killed resprouts',
       'not_fire_killed_does_not_resprout', 'fire_not_relevant',
       'fire_not_relevant resprouts',
       'fire_not_relevant not_fire_killed_does_not_resprout',
       'fire_killed fire_not_relevant'], dtype=object)

In [32]:
# ATtraits['trait_name'].unique()
ss = (ATtraits['trait_name']=='photosynthetic_pathway' )
ATtraits[ss]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
175021,Cunningham_1999,Acacia binervata,Cunningham_Knights Hill_665m,NaN,Cunningham_1999_01,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Acacia binervata
175037,Cunningham_1999,Acacia brachybotrya,Cunningham_Nombinnie_160m,NaN,Cunningham_1999_02,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Acacia brachybotrya
175052,Cunningham_1999,Acacia rigens,Cunningham_Rankins-Springs_170m,NaN,Cunningham_1999_03,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Acacia rigens
175068,Cunningham_1999,Acacia stricta,Cunningham_Narooma_25m,NaN,Cunningham_1999_04,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Acacia stricta
175083,Cunningham_1999,Boronia ledifolia,Cunningham_KNP-Waratah_165m,NaN,Cunningham_1999_05,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Boronia ledifolia
...,...,...,...,...,...,...,...,...,...,...,...,...
969325,Williams_2011,Austrostipa muelleri,NaN,grows in Adelaide,Williams_2011_947,photosynthetic_pathway,c4,NaN,NaN,expert_mean,NaN,Austrostipa muelleri
969334,Williams_2011,Deyeuxia minor,NaN,grows in Adelaide,Williams_2011_948,photosynthetic_pathway,c4,NaN,NaN,expert_mean,NaN,Deyeuxia minor
969360,Williams_2011,Aphanes australiana,NaN,grows in Adelaide,Williams_2011_951,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Aphanes australiana
969371,Williams_2011,Lomandra filiformis,NaN,grows in Adelaide,Williams_2011_952,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Lomandra filiformis


### Dormancy type

In [33]:
ss = (ATtraits['trait_name']=='dormancy_type' )
ATtraits[ss]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
479894,Ooi_2007,Acacia binervata,Fredericktown,NaN,Ooi_2007_00001,dormancy_type,physical_dormancy,NaN,1978-11-21,expert_mean,NaN,Acacia binervata
481466,Ooi_2007,Angophora bakeri,Agnes Banks to Castlereagh,NaN,Ooi_2007_01580,dormancy_type,non_dormant,NaN,1977-02-18,expert_mean,NaN,Angophora bakeri
488008,Ooi_2007,Isopogon anemonifolius,Cordeaux Cataract Catchment,NaN,Ooi_2007_08122,dormancy_type,physiological_dormancy,NaN,1975-08-12,expert_mean,NaN,Isopogon anemonifolius
489445,Ooi_2007,Bulbine bulbosa,Blacktown,NaN,Ooi_2007_09558,dormancy_type,morphophysiological_dormancy,NaN,1974-12-04,expert_mean,NaN,Bulbine bulbosa
489745,Ooi_2007,Calotis cuneifolia,Ashford,NaN,Ooi_2007_09858,dormancy_type,non_dormant physiological_dormancy,NaN,1978-11-25,expert_mean,NaN,Calotis cuneifolia


## Start XML file here